In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import wavelet_transform as wt
import signal_processing as sp
import scipy.sparse
from sklearn.preprocessing import StandardScaler
from pygsp import graphs

/tmp/ipykernel_21349/4231272995.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def precompute_wavelet(poly_division, time_interval):
    df = pd.read_csv(f"data/polygon_data/{poly_division}_{time_interval}.csv")
    features = df.columns[2:]
    try:
        adj_matrix = np.load(f"data/adj_matrix/{poly_division}.npy")
    except:
        adj_matrix = scipy.sparse.load_npz(f"data/adj_matrix/{poly_division}.npz").toarray()

    signal_multi = np.stack(df.groupby("id_poly").apply(lambda x: x.sort_values("date").drop(columns=["id_poly", "date"]).values).values)
    
    df_res = []

    for k, feature in enumerate(features):
        df_start = df[["id_poly", "date"]].copy()
        signal = signal_multi[:, :, k]
        n_timestamps = signal_multi.shape[1]

        wav =  wt.WaveletTransform(
            adj_matrix,
            n_timestamps,
            graph_product="strong",
            n_filters=4,
            kernel="abspline",
            scaling_function=False,
            method="chebyshev",
            order_chebyshev=30,
        )
        coeffs = wav.transform(signal)
        coeffs = sp.get_scaled_coefficients(coeffs)


        # create column for coeffs in the df_res
        for i in range(coeffs.shape[-1]):
            df_start[f"mean_freq_{i}"] = coeffs[:, :, i].flatten()
        df_start["type"] = feature

        df_res.append(df_start)

    df_res = pd.concat(df_res)
    float_columns = df_res.columns[df_res.dtypes == np.float64] 
    df_res[float_columns] = df_res[float_columns].astype(np.float16)
    df_res.to_csv(f"data/coeffs/{poly_division}_{time_interval}.csv", index=False)

In [3]:
def create_G_H_spatial(adjancency_matrix, n_timestamps):

    x, y = np.where(adjancency_matrix)
    e = []
    nodes = np.arange(adjancency_matrix.shape[0])
    n_nodes = adjancency_matrix.shape[0]
    
    for t in range(n_timestamps):  # link between nodes in the same timestamp
        e.append(np.array([x + t * n_nodes, y + t * n_nodes]).T)

    e = np.concatenate(e)
    adjancency_matrix_temporal = scipy.sparse.csr_matrix(
        (np.ones(e.shape[0]), (e[:, 0], e[:, 1])),
        shape=(
            n_timestamps * adjancency_matrix.shape[0],
            n_timestamps * adjancency_matrix.shape[0],
        ),
    )
    G_H = graphs.Graph(adjancency_matrix_temporal)
    return G_H


def precompute_spatial_wavelet(poly_division, time_interval):
    
    df = pd.read_csv(f"data/polygon_data/{poly_division}_{time_interval}.csv")
    features = df.columns[2:]
    try:
        adj_matrix = np.load(f"data/adj_matrix/{poly_division}.npy")
    except:
        adj_matrix = scipy.sparse.load_npz(f"data/adj_matrix/{poly_division}.npz").toarray()
    signal_multi = np.stack(df.groupby("id_poly").apply(lambda x: x.sort_values("date").drop(columns=["id_poly", "date"]).values).values)

    n_timestamps = signal_multi.shape[1]
    G_H = create_G_H_spatial(adj_matrix, n_timestamps)
    df_res = []

    for k, feature in enumerate(features):
        df_start = df[["id_poly", "date"]].copy()
        signal = signal_multi[:, :, k]

        n_timestamps = signal_multi.shape[1]

        wav =  wt.WaveletTransform(
            adj_matrix,
            n_timestamps,
            G_H=G_H,
            graph_product="strong",
            n_filters=4,
            kernel="abspline",
            scaling_function=False,
            method="chebyshev",
            order_chebyshev=30,
        )
        coeffs = wav.transform(signal)
        coeffs = sp.get_scaled_coefficients(coeffs)
        # create column for coeffs in the df_res
        for i in range(coeffs.shape[-1]):
            df_start[f"mean_freq_{i}"] = coeffs[:, :, i].flatten()
        df_start["type"] = feature
        df_res.append(df_start)
        
    df_res = pd.concat(df_res)
    float_columns = df_res.columns[df_res.dtypes == np.float64] 
    df_res[float_columns] = df_res[float_columns].astype(np.float16)
    df_res.to_csv(f"data/coeffs_spatial/{poly_division}_{time_interval}.csv", index=False)
    

In [4]:
def precompute_both(poly_division, time_interval):
    precompute_wavelet(poly_division, time_interval)
    precompute_spatial_wavelet(poly_division, time_interval)

In [5]:
precompute_both("SpCenterCensus5k", "Period1")


/tmp/ipykernel_21349/4064696322.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  signal_multi = np.stack(df.groupby("id_poly").apply(lambda x: x.sort_values("date").drop(columns=["id_poly", "date"]).values).values)
/tmp/ipykernel_21349/859572568.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  signal_multi = np.stack(df.groupby("id_poly").apply(lambda x: x.sort_values("date").drop(columns=["id_poly",

In [6]:
precompute_both("SpCenterCensus5k", "Period2")

/tmp/ipykernel_21349/4064696322.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  signal_multi = np.stack(df.groupby("id_poly").apply(lambda x: x.sort_values("date").drop(columns=["id_poly", "date"]).values).values)
/tmp/ipykernel_21349/859572568.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  signal_multi = np.stack(df.groupby("id_poly").apply(lambda x: x.sort_values("date").drop(columns=["id_poly",

In [7]:
precompute_both("SpCenterCensus2k", "Period1")

/tmp/ipykernel_21349/4064696322.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  signal_multi = np.stack(df.groupby("id_poly").apply(lambda x: x.sort_values("date").drop(columns=["id_poly", "date"]).values).values)
/tmp/ipykernel_21349/859572568.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  signal_multi = np.stack(df.groupby("id_poly").apply(lambda x: x.sort_values("date").drop(columns=["id_poly",

In [8]:
precompute_both("SpCenterCensus2k", "Period2")

/tmp/ipykernel_21349/4064696322.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  signal_multi = np.stack(df.groupby("id_poly").apply(lambda x: x.sort_values("date").drop(columns=["id_poly", "date"]).values).values)


In [ ]:
precompute_both("NYBlocks", "Period1")

/tmp/ipykernel_752869/4139791210.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  signal_multi = np.stack(df.groupby("id_poly").apply(lambda x: x.sort_values("date").drop(columns=["id_poly", "date"]).values).values)
/tmp/ipykernel_752869/2335242727.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  signal_multi = np.stack(df.groupby("id_poly").apply(lambda x: x.sort_values("date").drop(columns=["id_pol

In [ ]:
precompute_both("BLACities", "Year")

/tmp/ipykernel_746325/4139791210.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  signal_multi = np.stack(df.groupby("id_poly").apply(lambda x: x.sort_values("date").drop(columns=["id_poly", "date"]).values).values)
/tmp/ipykernel_746325/2335242727.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  signal_multi = np.stack(df.groupby("id_poly").apply(lambda x: x.sort_values("date").drop(columns=["id_pol

In [ ]:
precompute_both("BLACitiesDiff", "Year")